In [41]:
from lxml import html  
import csv,os,json
import requests
import pandas as pd
from time import sleep
#import socket
from selenium import webdriver
#import socks

In [51]:
#socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
#socket.socket = socks.socksocket
#driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
#driver = webdriver.Chrome(executable_path=r'C:\Geckodriver\chromedriver.exe')
#driver.set_page_load_timeout(100)
#print(requests.get("http://icanhazip.com").text)
#print(driver.get("http://icanhazip.com").text)
#headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
driver.set_page_load_timeout(100)

In [52]:
def AmzonParser(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    driver.get(url)
    #print(driver)
    rpage = requests.get(url,headers=headers)
    page = driver.page_source
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page)
            XPATH_NAME = '//h1[@id="title"]//text()'
            #XPATH_SALE_PRICE = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            XPATH_SALE_PRICE = '//span[contains(@class,"offer-price") and contains(@class,"a-size-medium")]//text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()' 
            XPATH_AUTHOR = '//a[contains(@class,"contributorNameID")]//text()'
            XPATH_ISBN10 = '//li/b[contains(text(),"ISBN-10:")]/following-sibling::text()'
            XPATH_PAGES = '//li/b[contains(text(),"Paperback") or contains(text(),"Hardcover") or contains(text(),"Print Length")]/following-sibling::text()'
            XPATH_PUBLISHER = '//li/b[contains(text(),"Publisher")]/following-sibling::text()'
            XPATH_LANGUAGE = '//li/b[contains(text(),"Language")]/following-sibling::text()'
            XPATH_DIMENSIONS = '//li/b[contains(text(),"Dimensions")]/following-sibling::text()'
            XPATH_WEIGHT = '//li/b[contains(text(),"Weight")]/following-sibling::text()'
            XPATH_RATING = '//li/b[contains(text(),"Average Customer Review:")]/following-sibling::span/span/a/i/span/text()'
           
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            ##RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            RAW_AUTHOR = doc.xpath(XPATH_AUTHOR)
            RAW_ISBN10 = doc.xpath(XPATH_ISBN10)
            RAW_PAGES = doc.xpath(XPATH_PAGES)
            RAW_PUBLISHER = doc.xpath(XPATH_PUBLISHER) 
            RAW_LANGUAGE = doc.xpath(XPATH_LANGUAGE)
            RAW_DIMENSIONS = doc.xpath(XPATH_DIMENSIONS)
            RAW_WEIGHT = doc.xpath(XPATH_WEIGHT)
            RAW_RATING = doc.xpath(XPATH_RATING)
            
            print(RAW_RATING)
 
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            #ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAw_AVAILABILITY).strip() if RAw_AVAILABILITY else None
            AUTHOR = ''.join(RAW_AUTHOR).strip() if RAW_AUTHOR else None
            ISBN10 = ''.join(RAW_ISBN10).strip() if RAW_ISBN10 else None
            PAGES = ''.join(RAW_PAGES).strip() if RAW_PAGES else None
            PUBLISHER = ''.join(RAW_PUBLISHER).strip() if RAW_PUBLISHER else None
            LANGUAGE = ''.join(RAW_LANGUAGE).strip() if RAW_LANGUAGE else None
            DIMENSIONS = ''.join(RAW_DIMENSIONS).strip() if RAW_DIMENSIONS else None
            WEIGHT = ''.join(RAW_WEIGHT).strip().strip('()').strip() if RAW_WEIGHT else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            #if not ORIGINAL_PRICE:
                #ORIGINAL_PRICE = SALE_PRICE
 
            if rpage.status_code!=200 or not NAME or NAME == "":
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    #'ORIGINAL_PRICE':ORIGINAL_PRICE,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'AUTHOR' : AUTHOR,
                    'ISBN10' : ISBN10,
                    'PAGES' : PAGES,
                    'PUBLISHER' : PUBLISHER,
                    'LANGUAGE' : LANGUAGE,
                    'DIMENSIONS' : DIMENSIONS,
                    'WEIGHT' : WEIGHT,
                    'RATING' : RATING,
                    }
 
            return data
        except Exception as e:
            print(e)
            driver.get(url)
            #print(driver)
            rpage = requests.get(url,headers=headers)
            page = driver.page_source
 

In [53]:
extracted_data = []
def ReadAsin(AsinList):
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    count = 0
    for i in AsinList:
        url = "http://www.amazon.com/gp/product/"+i
        print("Processing: "+url)
        extracted_data.append(AmzonParser(url))
        count += 1
        #sleep(5)
        print(count)
    f=open('data.json','w')
    json.dump(extracted_data,f,indent=4)
 

In [54]:
formatted_asin = []
df = pd.read_csv("amazon_product_asins_rom.csv")
for index, row in df.iterrows():
    if len(row[0]) < 8:
        asin = "000" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 9:
        asin = "00" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 10:
        asin = "0" + row[0]
        formatted_asin.append(asin)
    else:
        formatted_asin.append(row[0])

In [55]:
if __name__ == "__main__":
    ReadAsin(formatted_asin[420:1000])

Processing: http://www.amazon.com/gp/product/0399583939
['4.7 out of 5 stars']
1
Processing: http://www.amazon.com/gp/product/1616204516
['4.4 out of 5 stars']
2
Processing: http://www.amazon.com/gp/product/1455540773
['4.5 out of 5 stars']
3
Processing: http://www.amazon.com/gp/product/0147517729
['4.6 out of 5 stars']
4
Processing: http://www.amazon.com/gp/product/031631806X
['3.8 out of 5 stars']
5
Processing: http://www.amazon.com/gp/product/0062371916
['4.6 out of 5 stars']
6
Processing: http://www.amazon.com/gp/product/0446547638
['4.5 out of 5 stars']
7
Processing: http://www.amazon.com/gp/product/0515151505
['4.7 out of 5 stars']
8
Processing: http://www.amazon.com/gp/product/1416563717
['4.5 out of 5 stars']
9
Processing: http://www.amazon.com/gp/product/0060591838
['4.4 out of 5 stars']
10
Processing: http://www.amazon.com/gp/product/1594634025
['4.1 out of 5 stars']
11
Processing: http://www.amazon.com/gp/product/0062645293
[]
12
Processing: http://www.amazon.com/gp/product/

['4.4 out of 5 stars']
103
Processing: http://www.amazon.com/gp/product/1491089288
['4.5 out of 5 stars']
104
Processing: http://www.amazon.com/gp/product/0143124544
['4.6 out of 5 stars']
105
Processing: http://www.amazon.com/gp/product/0307387267
['4.6 out of 5 stars']
106
Processing: http://www.amazon.com/gp/product/0441020151
['3.9 out of 5 stars']
107
Processing: http://www.amazon.com/gp/product/1250161533
['4.5 out of 5 stars']
108
Processing: http://www.amazon.com/gp/product/1476775850
['4.4 out of 5 stars']
109
Processing: http://www.amazon.com/gp/product/0062349066
['4.7 out of 5 stars']
110
Processing: http://www.amazon.com/gp/product/0515156124
['4.7 out of 5 stars']
111
Processing: http://www.amazon.com/gp/product/1936399008
['4.5 out of 5 stars']
112
Processing: http://www.amazon.com/gp/product/1451687605
['4.8 out of 5 stars']
113
Processing: http://www.amazon.com/gp/product/1490505776
['4.6 out of 5 stars']
114
Processing: http://www.amazon.com/gp/product/1481448536
['4.

['4.5 out of 5 stars']
206
Processing: http://www.amazon.com/gp/product/1925559076
['4.7 out of 5 stars']
207
Processing: http://www.amazon.com/gp/product/0306821990
['3.8 out of 5 stars']
208
Processing: http://www.amazon.com/gp/product/1250074355
['4.4 out of 5 stars']
209
Processing: http://www.amazon.com/gp/product/1946660302
['4.8 out of 5 stars']
210
Processing: http://www.amazon.com/gp/product/1847492126
['5.0 out of 5 stars']
211
Processing: http://www.amazon.com/gp/product/1538744880
[]
212
Processing: http://www.amazon.com/gp/product/0373282494
['4.1 out of 5 stars']
213
Processing: http://www.amazon.com/gp/product/1496712528
['4.5 out of 5 stars']
214
Processing: http://www.amazon.com/gp/product/142014314X
['5.0 out of 5 stars']
215
Processing: http://www.amazon.com/gp/product/0999152904
['4.4 out of 5 stars']
216
Processing: http://www.amazon.com/gp/product/0425263916
['4.6 out of 5 stars']
217
Processing: http://www.amazon.com/gp/product/1420145355
['4.0 out of 5 stars']
2

['4.3 out of 5 stars']
308
Processing: http://www.amazon.com/gp/product/0399564519
[]
309
Processing: http://www.amazon.com/gp/product/1414333005
['4.8 out of 5 stars']
310
Processing: http://www.amazon.com/gp/product/0399585168
['4.0 out of 5 stars']
311
Processing: http://www.amazon.com/gp/product/0451417089
['4.7 out of 5 stars']
312
Processing: http://www.amazon.com/gp/product/0062371223
['4.4 out of 5 stars']
313
Processing: http://www.amazon.com/gp/product/1496707192
['4.7 out of 5 stars']
314
Processing: http://www.amazon.com/gp/product/1480118516
['4.2 out of 5 stars']
315
Processing: http://www.amazon.com/gp/product/1979159092
['5.0 out of 5 stars']
316
Processing: http://www.amazon.com/gp/product/1476746303
['4.3 out of 5 stars']
317
Processing: http://www.amazon.com/gp/product/0615788084
['4.3 out of 5 stars']
318
Processing: http://www.amazon.com/gp/product/0143126474
['4.0 out of 5 stars']
319
Processing: http://www.amazon.com/gp/product/042524413X
['4.6 out of 5 stars']
3

['4.7 out of 5 stars']
410
Processing: http://www.amazon.com/gp/product/0446619280
['4.0 out of 5 stars']
411
Processing: http://www.amazon.com/gp/product/1503939790
['4.5 out of 5 stars']
412
Processing: http://www.amazon.com/gp/product/0373804059
[]
413
Processing: http://www.amazon.com/gp/product/1101872241
['3.4 out of 5 stars']
414
Processing: http://www.amazon.com/gp/product/1335526226
['4.5 out of 5 stars']
415
Processing: http://www.amazon.com/gp/product/1416598367
['3.8 out of 5 stars']
416
Processing: http://www.amazon.com/gp/product/1947222279
['4.9 out of 5 stars']
417
Processing: http://www.amazon.com/gp/product/1454902493
['4.4 out of 5 stars']
418
Processing: http://www.amazon.com/gp/product/0515138681
['4.7 out of 5 stars']
419
Processing: http://www.amazon.com/gp/product/0692055231
['4.8 out of 5 stars']
420
Processing: http://www.amazon.com/gp/product/1503951731
['4.8 out of 5 stars']
421
Processing: http://www.amazon.com/gp/product/0345545958
['4.4 out of 5 stars']
4

[]
captha
['4.4 out of 5 stars']
509
Processing: http://www.amazon.com/gp/product/0692068252
['5.0 out of 5 stars']
510
Processing: http://www.amazon.com/gp/product/1451687524
[]
captha
['4.9 out of 5 stars']
511
Processing: http://www.amazon.com/gp/product/1501165836
['4.4 out of 5 stars']
512
Processing: http://www.amazon.com/gp/product/1943893047
[]
captha
['4.6 out of 5 stars']
513
Processing: http://www.amazon.com/gp/product/0425281396
[]
514
Processing: http://www.amazon.com/gp/product/0062371851
[]
captha
['4.5 out of 5 stars']
515
Processing: http://www.amazon.com/gp/product/0768420164
[]
captha
['4.6 out of 5 stars']
516
Processing: http://www.amazon.com/gp/product/1979825297
[]
captha
['4.5 out of 5 stars']
517
Processing: http://www.amazon.com/gp/product/1496434293
[]
518
Processing: http://www.amazon.com/gp/product/1503934861
['4.7 out of 5 stars']
519
Processing: http://www.amazon.com/gp/product/0425260925
['4.6 out of 5 stars']
520
Processing: http://www.amazon.com/gp/pro